<a href="https://colab.research.google.com/github/luciacasass/UFV-VisualizacionDatos/blob/main/EjerciciosClase/Clase_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Visualización de Datos - Clase 2

Lucía Casas Sierra

### Exploración inicial

In [1]:
# Importación de librerías
import pandas as pd
import numpy as np
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")


En el siguiente informe, se presenta una base de datos que contiene información sobre las ventas registradas para diferentes submarcas en distintos países de Europa a lo largo de una franja temporal. El objetivo principal del estudio es la identificación del comportamiento de las predicciones a partir de diferentes herramientas de representación gráfica. Para ello, se procederá a la carga de los datos y las transformaciones adecuadas de los campos para su procesamiento.

In [2]:
# Carga de datos
df = pd.read_csv('datos_ejercicio_ventas.csv')
print(df.head())

countries = df['COUNTRY'].unique()
brands = df['SUBBRAND'].unique()

# Cálculo del tiempo que abarca cada escenario: ventas reales y predicciones
df['DATE'] = pd.to_datetime(df['YEAR'].astype(str) + '-' + df['MONTH'].astype(str) + '-01')
date_range_actual = [df['DATE'][df['SCENARIO']=='actual'].dt.date.min().strftime('%Y-%m'),
                     df['DATE'][df['SCENARIO']=='actual'].dt.date.max().strftime('%Y-%m')]
date_range_forecast = [df['DATE'][df['SCENARIO']=='AI_forecast'].dt.date.min().strftime('%Y-%m'),
                       df['DATE'][df['SCENARIO']=='AI_forecast'].dt.date.max().strftime('%Y-%m')]

print('\nLista de países:\n', countries)
print('\nLista de marcas:\n', brands)
print()
print('Espacio temporal para datos reales de ventas: ', date_range_actual)
print('Espacio temporal para predicciones de ventas: ', date_range_forecast)

         COUNTRY        SUBBRAND  YEAR  MONTH     SCENARIO FORECAST  \
0       Portugal     Lipton (L3)  2023     12  AI_forecast  AI_P02F   
1  Great Britain     Lipton (L3)  2023     12  AI_forecast  AI_P10F   
2          Spain  Pepsi Max (L3)  2023     12  AI_forecast  AI_P09F   
3  Great Britain        7up (L3)  2024     12  AI_forecast  AI_P10F   
4        Hungary     Lipton (L3)  2023      9  AI_forecast  AI_P03F   

   FORECAST_YEAR         AMOUNT  
0         2023.0  754356.237194  
1         2023.0  560030.558029  
2         2023.0   88501.980847  
3         2023.0  363224.511516  
4         2023.0  396176.120491  

Lista de países:
 ['Portugal' 'Great Britain' 'Spain' 'Hungary' 'Norway' 'Denmark'
 'Netherlands' 'Italy' 'Czech']

Lista de marcas:
 ['Lipton (L3)' 'Pepsi Max (L3)' '7up (L3)' 'Pepsi Regular (L3)'
 'Mountain Dew (L3)' '7up Free (L3)']

Espacio temporal para datos reales de ventas:  ['2023-01', '2024-08']
Espacio temporal para predicciones de ventas:  ['2023-01', '2

A continuación, se analizará la información relativa a los valores de ventas: se evaluará la proporción de datos conocidos y cifras predecidas, y la viabilidad de la presencia de outliers en el resto del estudio .

In [3]:
# Resumen Ventas Reales vs Predicciones
df_actual = df[df['SCENARIO'] == 'actual']
df_forecast = df[df['SCENARIO'] == 'AI_forecast']

print('VENTAS REALES:')
print('Número de filas: ', len(df_actual))
print(f'Representa el {(len(df_actual)/len(df)*100):.2f}% del archivo original')

print('\nDATOS DE PRONÓSTICO:')
print('Número de filas: ', len(df_forecast))
print(f'Representa el {(len(df_forecast)/len(df)*100):.2f}% del archivo original')

VENTAS REALES:
Número de filas:  900
Representa el 4.82% del archivo original

DATOS DE PRONÓSTICO:
Número de filas:  17766
Representa el 95.18% del archivo original


Para la evaluación de los valores del DataFrame de registros reales, primero se obtendrá un resumen estadístico del comportamiento según la marca del producto.


In [4]:
# Resumen estadístico
stats_df = df_actual.groupby(['SUBBRAND'])['AMOUNT'].agg(
    mean='mean',
    std_dev='std',
    median='median',
    min='min',
    max='max',
    q25=lambda x: x.quantile(0.25),
    q75=lambda x: x.quantile(0.75),
    count='count'
).reset_index()

print(stats_df)

             SUBBRAND          mean       std_dev         median  \
0            7up (L3)  1.203110e+05  1.402592e+05   75096.547801   
1       7up Free (L3)  2.331721e+05  4.029420e+05   78712.619670   
2         Lipton (L3)  5.318624e+05  5.459570e+05  330685.544633   
3   Mountain Dew (L3)  7.474391e+04  6.212235e+04   54549.290082   
4      Pepsi Max (L3)  2.203882e+06  3.788145e+06  487750.741443   
5  Pepsi Regular (L3)  7.768370e+05  7.250352e+05  422975.358041   

             min           max            q25           q75  count  
0  -19481.652378  6.015346e+05   11022.415870  1.592700e+05    156  
1      89.354912  1.533494e+06   43560.456630  1.366306e+05    147  
2 -217120.103133  2.411276e+06   15308.024487  8.292715e+05    137  
3    8073.094616  2.323776e+05   31081.687249  9.120060e+04    100  
4   80772.495122  1.481563e+07  210154.618078  2.264963e+06    180  
5   34297.395969  2.439380e+06  183688.229438  1.221954e+06    180  


Se observa que algunos de los campos muestran valores negativos. Conociendo el dominio del estudio, se excluirán estos registros de la base de datos final, ya que presentan comportamientos demasiado atípicos. Además, se eliminarán las columnas que en el DataFrame original hacían referencia a campos atributos exclusivos de los escenarios de predicciones.

In [5]:
# Eliminar valores negativos de df_actual
df_actual = df_actual[df_actual['AMOUNT'] > 0]
print('Número de registros en Actuals: ', len(df_actual))

# Eliminar columnas que no interesan
df_actual = df_actual.drop(columns=['FORECAST', 'FORECAST_YEAR'])

Número de registros en Actuals:  890


In [6]:
# Comprobar si hay valores nulos
null_counts = df_actual.isnull().sum()

print(null_counts)

COUNTRY     0
SUBBRAND    0
YEAR        0
MONTH       0
SCENARIO    0
AMOUNT      0
DATE        0
dtype: int64


Una vez se ha finalizado la primera inspección del DataFrame de los *Actuals*, se repite el mismo proceso para los datos de *Forecast*.

In [7]:
# Resumen estadístico
stats_df = df_forecast.groupby(['SUBBRAND'])['AMOUNT'].agg(
    mean='mean',
    std_dev='std',
    median='median',
    min='min',
    max='max',
    q25=lambda x: x.quantile(0.25),
    q75=lambda x: x.quantile(0.75),
    count='count'
).reset_index()

print(stats_df)

             SUBBRAND          mean       std_dev         median  \
0            7up (L3)  1.075674e+05  1.268640e+05   63535.376374   
1       7up Free (L3)  4.700942e+05  6.218755e+05  114672.402313   
2         Lipton (L3)  6.295630e+05  5.486773e+05  560213.339785   
3   Mountain Dew (L3)  7.752728e+04  7.291431e+04   47115.096998   
4      Pepsi Max (L3)  2.196140e+06  3.031393e+06  991225.151914   
5  Pepsi Regular (L3)  7.347977e+05  6.291315e+05  558913.258971   

            min           max            q25           q75  count  
0      0.000000  6.654884e+05   11108.087904  1.526276e+05   2538  
1     13.736941  9.750512e+06   49990.802335  1.010236e+06   2196  
2      0.000000  2.549642e+06  247696.897661  7.911472e+05   2556  
3      0.000000  2.693993e+05   10969.049856  1.433676e+05   1602  
4  61568.314722  1.373810e+07  406029.562025  2.511763e+06   5418  
5  12997.861011  2.526327e+06  165409.577382  1.191550e+06   3456  


In [8]:
# Eliminar registros de predicciones nulas
df_forecast = df_forecast[df_forecast['AMOUNT'] > 0]
print('Número de registros en Forecast: ', len(df_forecast))

Número de registros en Forecast:  17030


In [9]:
# Columnas con valores nulos
null_counts = df_forecast.isnull().sum()

print(null_counts)

COUNTRY          0
SUBBRAND         0
YEAR             0
MONTH            0
SCENARIO         0
FORECAST         0
FORECAST_YEAR    0
AMOUNT           0
DATE             0
dtype: int64


Tras los cambios aplicados al contenido de la base de datos original, se actualiza la distribución de registros en función de la naturaleza del escenario.

In [10]:
print('VENTAS REALES:')
print('Número de filas: ', len(df_actual))
print(f'Representa el {(len(df_actual)/(len(df_actual)+len(df_forecast))*100):.2f}% del archivo original')

print('\nDATOS DE PRONÓSTICO:')
print('Número de filas: ', len(df_forecast))
print(f'Representa el {(len(df_forecast)/(len(df_actual)+len(df_forecast))*100):.2f}% del archivo original')

VENTAS REALES:
Número de filas:  890
Representa el 4.97% del archivo original

DATOS DE PRONÓSTICO:
Número de filas:  17030
Representa el 95.03% del archivo original


#### Horizontes de predicción

Se procede al cálculo del horizonte de predicción para los datos de pronóstico disponibles para cada marca por país. Este indicador permite comprender la extensión máxima que adoptan las predicciones. De esta forma, se obtiene la mayor diferencia temporal entre la fecha en la que se lanza la predicción y la fecha sobre la que está prediciendo.

In [11]:
# Se extrae el mes en el que se registró la predicción
def horizon_months(df):
  df['FORECAST_MONTH'] = pd.to_numeric(df['FORECAST'].str.extract('-(\d+)')[0])
  # Los campos que no contienen información numérica se refieren a las predicciones registradas en el primer mes
  df['FORECAST_MONTH'] = df['FORECAST_MONTH'].fillna(1).astype(int)
  df['FORECAST_YEAR'] = df['FORECAST_YEAR'].astype(int)

  # Se unen en una sola columna con la fecha completa
  df['FORECAST_DATE'] = pd.to_datetime(
      df['FORECAST_YEAR'].astype(str) + '-' +
      df['FORECAST_MONTH'].astype(str) + '-01'
  )
  # De acuerdo con el conocimiento de la base de datos, se resta un mes
  df['FORECAST_DATE'] = df['FORECAST_DATE'] - pd.DateOffset(months=1)

  # Horizonte (meses) = fecha para predicción - fecha de registro
  df['HORIZON_MONTHS'] = (
      (df['DATE'].dt.year - df['FORECAST_DATE'].dt.year) * 12
      + (df['DATE'].dt.month - df['FORECAST_DATE'].dt.month)
  )
  return df

df_forecast = horizon_months(df_forecast)

print(df_forecast.head())


         COUNTRY        SUBBRAND  YEAR  MONTH     SCENARIO FORECAST  \
0       Portugal     Lipton (L3)  2023     12  AI_forecast  AI_P02F   
1  Great Britain     Lipton (L3)  2023     12  AI_forecast  AI_P10F   
2          Spain  Pepsi Max (L3)  2023     12  AI_forecast  AI_P09F   
3  Great Britain        7up (L3)  2024     12  AI_forecast  AI_P10F   
4        Hungary     Lipton (L3)  2023      9  AI_forecast  AI_P03F   

   FORECAST_YEAR         AMOUNT       DATE  FORECAST_MONTH FORECAST_DATE  \
0           2023  754356.237194 2023-12-01               1    2022-12-01   
1           2023  560030.558029 2023-12-01               1    2022-12-01   
2           2023   88501.980847 2023-12-01               1    2022-12-01   
3           2023  363224.511516 2024-12-01               1    2022-12-01   
4           2023  396176.120491 2023-09-01               1    2022-12-01   

   HORIZON_MONTHS  
0              12  
1              12  
2              12  
3              24  
4               

In [12]:
# Extracción del horizonte máximo por cada país y submarca
subbrand_horizon_months = df_forecast.groupby(['COUNTRY', 'SUBBRAND'])['HORIZON_MONTHS'].max().reset_index()
print('Horizonte de predicción para los pares Country + Subbrand: ', subbrand_horizon_months['HORIZON_MONTHS'].unique())
print(subbrand_horizon_months)


Horizonte de predicción para los pares Country + Subbrand:  [29 25 27]
          COUNTRY            SUBBRAND  HORIZON_MONTHS
0           Czech            7up (L3)              29
1           Czech         Lipton (L3)              29
2           Czech   Mountain Dew (L3)              29
3           Czech      Pepsi Max (L3)              29
4           Czech  Pepsi Regular (L3)              29
5         Denmark       7up Free (L3)              29
6         Denmark   Mountain Dew (L3)              29
7         Denmark      Pepsi Max (L3)              29
8         Denmark  Pepsi Regular (L3)              29
9   Great Britain            7up (L3)              29
10  Great Britain       7up Free (L3)              29
11  Great Britain         Lipton (L3)              29
12  Great Britain   Mountain Dew (L3)              29
13  Great Britain      Pepsi Max (L3)              29
14  Great Britain  Pepsi Regular (L3)              29
15        Hungary            7up (L3)              25
16        H

De acuerdo a los resultados obtenidos, el horizonte de predicción se corresponde con 29 meses en la mayoría de los casos. No obstante, existen algunas excepciones que reducen el horizonte a 25 y 27 meses. Vamos a estudiar esos casos particulares para evaluar si se han visto reducidos como consecuencia de la limpieza de datos previa.

In [13]:
horizon_25 = subbrand_horizon_months[['COUNTRY', 'SUBBRAND']][subbrand_horizon_months['HORIZON_MONTHS'] == 25]
horizon_25 = list(horizon_25.iloc[0, :])
print(horizon_25)

horizon_27 = subbrand_horizon_months[['COUNTRY', 'SUBBRAND']][subbrand_horizon_months['HORIZON_MONTHS'] == 27]
horizon_27 = list(horizon_27.iloc[0, :])
print(horizon_27)

['Hungary', '7up (L3)']
['Norway', '7up (L3)']


In [14]:
# Cálculo del horizonte
df_25 = df[(df['SCENARIO']=='AI_forecast') & (df['COUNTRY'] == horizon_25[0]) & (df['SUBBRAND'] == horizon_25[1])]
df_27 = df[(df['SCENARIO']=='AI_forecast') & (df['COUNTRY'] == horizon_27[0]) & (df['SUBBRAND'] == horizon_27[1])]

df_25 = horizon_months(df_25)
df_27 = horizon_months(df_27)

In [15]:
# Extracción del horizonte máximo
df_25 = df_25.groupby(['COUNTRY', 'SUBBRAND'])['HORIZON_MONTHS'].max().reset_index()
print('Horizonte de predicción para',
      ', '.join(df_25['COUNTRY'].astype(str)),
      'en',
      ', '.join(df_25['SUBBRAND'].astype(str)),
      ':',
      ', '.join(df_25['HORIZON_MONTHS'].unique().astype(str)))

df_27 = df_27.groupby(['COUNTRY', 'SUBBRAND'])['HORIZON_MONTHS'].max().reset_index()
print('Horizonte de predicción para',
      ', '.join(df_27['COUNTRY'].astype(str)),
      'en',
      ', '.join(df_27['SUBBRAND'].astype(str)),
      ':',
      ', '.join(df_27['HORIZON_MONTHS'].unique().astype(str)))

Horizonte de predicción para Hungary en 7up (L3) : 29
Horizonte de predicción para Norway en 7up (L3) : 29


Así, se confirma que el horizonte de predicción para todos los casos de estudio únicos de **País + Submarca** son de 29 meses.

### Distribución de ventas reales
Una vez se ha llevado a cabo la primera adaptación y exploración de los datos de estudio, se continúa con la representación gráfica de la información con el fin de trazar los distintos perfiles de ventas en función del país de comercialización, la marca del producto y la línea temporal.
#### Por país
En este caso en concreto, se requiere la visualización de la comparativa del volumen de ventas entre países. Para ello, se recurre a su representación a partir del diagrama de barras apiladas.

In [16]:
# Agrupar por país, sumando el total de ventas
sales_by_country = df_actual.groupby('COUNTRY')['AMOUNT'].sum().reset_index()
sales_by_country.columns = ['COUNTRY', 'TOTAL_SALES']

# Crear el gráfico de barras apiladas
fig = px.bar(
    sales_by_country,
    x='COUNTRY',
    y='TOTAL_SALES',
    color_discrete_sequence=['#457b9d'],
    title='Total Sales by Country',
    labels={'TOTAL_SALES': 'Total Sales', 'COUNTRY': 'Country'}
)

fig.update_layout(template='plotly_white')
fig.show()

Asísmismo, se aprovecha esta característica de localización para graficar los datos sobre un mapa de calor para entender la distribución geográfica de las ventas. Se observa que, frente a los resultados representados anteriormente, Reino Unido es el país que más destaca, complicando la identificación de diferencias entre otros países a partir del color de relleno asignado.

In [17]:
# Crear el gráfico (choropleth) usando el código de país (ISO-3)
fig = px.choropleth(
    sales_by_country,
    locations="COUNTRY",
    locationmode="country names",  # Modo para nombres de países
    color="TOTAL_SALES",
    color_continuous_scale=[[0, '#cde7f4'], [1, '#457b9d']],
    title="Total Sales by Country",
    labels={'TOTAL_SALES': 'Total Sales', 'COUNTRY': 'Country'}
)

# Hacer zoom sobre el continente europeo
fig.update_geos(
    scope="europe",  # Muestra solo Europa
    showcoastlines=True,  # Mostrar costas
    coastlinecolor="Black",  # Color de las costas
    coastlinewidth=0.5,  # Ancho de las costas
)
fig.update_layout(template='plotly_white')
fig.show()

#### Por mes y año
En el caso de la evolución temporal de las ventas, se ha de elegir un gráfico diferente para su trazado, pues la cantidad de fechas para las que se tienen datos es demasiado grande para su repersentación en barras apiladas. Para garantizar la legibilidad del gráfico, es necesario elegir una alternativa más adecuada. Por ello, se empleará el gráfico de líneas.

In [18]:
# Agrupar por fecha (mes + año), sumando el total de ventas
sales_by_month_year = df_actual.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index()

# Renombrar columnas
sales_by_month_year.columns = ['YEAR', 'MONTH', 'TOTAL_SALES']

# Columna que almacene la conjunción de mes y año
sales_by_month_year['YEAR_MONTH'] = sales_by_month_year['YEAR'].astype(str) + '-' + sales_by_month_year['MONTH'].astype(str).str.zfill(2)

fig = px.line(
    sales_by_month_year,
    x='YEAR_MONTH',
    y='TOTAL_SALES',
    line_shape='linear',
    color_discrete_sequence=['#457b9d'],
    title='Total Sales by Year and Month',
    height=600,
    labels={'TOTAL_SALES': 'Total Sales', 'YEAR_MONTH': 'Year-Month'}
)
fig.update_layout(template='plotly_white')
fig.show()

A simple vista, parece no existir ningún tipo de patrón identificable, o tendencia.
#### Por submarca
Por último, se mostrará la distribución de las ventas totales en función de la submarca de bebida asignada. Para este caso, similar al de su representación por países, se utilizará de nuevo el diagrama de barras apiladas. Se observa que una de las categorías, Pepsi Max (L3), presenta ventas totales bastante superiores al resto de submarcas del estudio. La siguiente submarca en el ranking de ventas es Pepsi Regular (L3), luego se puede concluir que la marca Pepsi tiene mayor presencia en el mercado de bebidas presente.

In [19]:
# Agrupar por marca, sumando el total de ventas
sales_by_subbrand = df_actual.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()
sales_by_subbrand.columns = ['SUBBRAND', 'TOTAL_SALES']

fig = px.bar(
    sales_by_subbrand,
    x='SUBBRAND',
    y='TOTAL_SALES',
    color_discrete_sequence=['#457b9d'],
    title='Total Sales by Subbrand',
    labels={'TOTAL_SALES': 'Total Sales', 'SUBBRAND': 'Subbrand'}
)

fig.update_layout(template='plotly_white')
fig.show()

### Tendencia y estacionalidad
En el siguiente apartado, se estudiará la presencia de patrones relacionados con la temporalidad de los datos. Para ello, se analizarán los casos del país con menos ventas y la marca con más ventas.
#### Todas las ventas del país con menos ventas
El primer paso para estudiar la tendencia y estacionalidad pasa por la observación de los datos en la gráfica con el fin de identificar patrones.

In [20]:
# Identificación del país con menor número de ventas
min_sales_country = sales_by_country.loc[sales_by_country['TOTAL_SALES'].idxmin(), 'COUNTRY']
print('País con menor número de ventas: ', min_sales_country)

País con menor número de ventas:  Spain


In [21]:
sales_by_country_month = df_actual.groupby(['COUNTRY', pd.Grouper(key='DATE', freq='M')])['AMOUNT'].sum().reset_index()
sales_by_country_month.columns = ['COUNTRY', 'DATE', 'TOTAL_SALES']

# Filtrar por el país con ventas mínimas
country_data = sales_by_country_month[sales_by_country_month['COUNTRY'] == min_sales_country]

fig = px.line(country_data, x='DATE', y='TOTAL_SALES',
              labels={'TOTAL_SALES': 'Total Sales', 'DATE':'Date'},
              title=f'Total Sales in {min_sales_country}')

fig.update_traces(line=dict(color='#457b9d'))

fig.update_layout(template='plotly_white')
fig.show()


A primera vista, no parece mostrar tendencia o estacionalidad. Sin embargo, a pesar de la escasez de muestras en la línea temporal, el comportamiento desde febrero de 2023 hasta agosto de ese mismo año parece presentar el mismo perfil que la imagen desde febrero de 2024 hasta agosto del mismo año. Este rasgo podría indicar una estacionalidad anual. Para comprobarlo, se recurrirá al cálculo de la media móvil para una ventana de 12 meses.

In [22]:
# Calcular la media móvil para un ventana de doce meses (anual)
country_data['Rolling_Mean'] = country_data['TOTAL_SALES'].rolling(window=12).mean()

color_map = {'TOTAL_SALES': '#457b9d', 'Rolling_Mean': '#8ecae6'}

fig = px.line(country_data, x='DATE', y=['TOTAL_SALES', 'Rolling_Mean'],
              color_discrete_map=color_map)

fig.update_layout(
    title=f'Total Sales and Rolling Avg in {min_sales_country}',
    xaxis_title='Date',
    yaxis_title='Total Sales',
    legend=dict(title='Legend'),
    template='plotly_white')

fig.show()

Gracias a la información reflejada en el gráfico que se ha generado, se aprecia que la media móvil se mantiene estable a lo largo del tiempo.

Por lo tanto, se puede concluir que las ventas totales en España presentan ciclos anuales que se repiten a lo largo del tiempo. Además, el análisis de dichos ciclos carece de una tendencia ascendente o descendente, indicando que las ventas se mantienen estables año tras año.

#### La marca con más ventas

In [23]:
# Identificación del país con mayor número de ventas
sales_by_brand_month = df_actual.groupby(['SUBBRAND', pd.Grouper(key='DATE', freq='M')])['AMOUNT'].sum().reset_index()
sales_by_brand_month.columns = ['SUBBRAND', 'DATE', 'TOTAL_SALES']
max_sales_brand = sales_by_brand_month.loc[sales_by_brand_month['TOTAL_SALES'].idxmax(), 'SUBBRAND']
print('Marca con mayor número de ventas: ', max_sales_brand)

Marca con mayor número de ventas:  Pepsi Max (L3)


In [24]:
sales_by_brand_month = df_actual.groupby(['SUBBRAND', pd.Grouper(key='DATE', freq='M')])['AMOUNT'].sum().reset_index()
sales_by_brand_month.columns = ['SUBBRAND', 'DATE', 'TOTAL_SALES']

# Filtrar por la submarca con ventas máximas
brand_data = sales_by_brand_month[sales_by_brand_month['SUBBRAND'] == max_sales_brand]

fig = px.line(brand_data, x='DATE', y='TOTAL_SALES',
              labels={'TOTAL_SALES': 'Total Sales', 'DATE':'Date'},
              title=f"Total Sales in {max_sales_brand}",)

fig.update_traces(line=dict(color='#457b9d'))
fig.update_layout(template='plotly_white')
fig.show()


Se observan dos picos mínimos bastante dramáticos para el mes de febrero en los años 2023 y 2024. Siguiendo la misma estrategia que en el caso anterior, se comprobará la evolución de la media móvil aplicada a una ventana de doce meses para confirmar la existencia de una posible estacionalidad anual.

In [25]:
# Calcular la media móvil para un ventana de doce meses (anual)
brand_data['Rolling_Mean'] = brand_data['TOTAL_SALES'].rolling(window=12).mean()

color_map = {'TOTAL_SALES': '#457b9d', 'Rolling_Mean': '#8ecae6'}

fig = px.line(brand_data, x='DATE', y=['TOTAL_SALES', 'Rolling_Mean'],
              color_discrete_map=color_map)

fig.update_layout(
    title=f'Total Sales and Rolling Avg for {max_sales_brand}',
    xaxis_title='Date',
    yaxis_title='Total Sales',
    legend=dict(title='Legend'),
    template='plotly_white')
fig.show()

En esta ocasión, la media móvil no presenta fluctuaciones. Además, mientras que en el caso de España la media se mantenía estable entorno a un valor, esta vez parece mantener una ligera tendencia a la alza.

Consecuentemente, se puede afirmar que las ventas totales de Pepsi en Europa presentan una estacionalidad anual, así como una tendencia creciente. Por lo tanto, a pesar de presentar el mismo perfil cada año, se prevee que las ventas de Pepsi aumenten con el tiempo.

### Predicciones España y evaluación de la bondad
Por último, se estudiará la bondad que muestran las predicciones realizadas en diferentes momentos en el tiempo en relación con los datos reales para el caso particular de ventas en España. El primer paso se centra en la visualización de la distribución de las predicciones frente al dato real. Para ello, se recurrirá al empleo del diagrama de bigotes o *boxplot*.

Se ha decidido analizar individualmente el comportamiento de las predicciones para cada una de las submarcas comercializadas en el país.

In [26]:
forecast_spain = df_forecast[df_forecast['COUNTRY'] == 'Spain']
actual_spain = df_actual[df_actual['COUNTRY'] == 'Spain']

Se comprueba si existen fechas para futuras predicciones y, por lo tanto, no tienen datos reales con los que compararlas. Estos casos se excluirán de la evaluación de bondad por su incompatibilidad con el objetivo.

In [27]:
set1 = set(actual_spain['DATE'].unique())
set2 = set(forecast_spain['DATE'].unique())

solo_en_lista1 = set1 - set2
solo_en_lista2 = set2 - set1

print("Elementos solo en lista1:", solo_en_lista1)
print("Elementos solo en lista2:", solo_en_lista2)

Elementos solo en lista1: set()
Elementos solo en lista2: {Timestamp('2024-10-01 00:00:00'), Timestamp('2024-12-01 00:00:00'), Timestamp('2025-02-01 00:00:00'), Timestamp('2025-03-01 00:00:00'), Timestamp('2024-11-01 00:00:00'), Timestamp('2025-05-01 00:00:00'), Timestamp('2024-09-01 00:00:00'), Timestamp('2025-01-01 00:00:00'), Timestamp('2025-04-01 00:00:00')}


In [28]:
# Estudiar sólo para las fechas de las que se conocen las ventas reales
combined_data = forecast_spain[forecast_spain['DATE'].isin(actual_spain['DATE'])].reset_index(drop=True)

# Crear una lista de gráficos por cada marca
for brand in combined_data['SUBBRAND'].unique():
    # Filtrar datos para la marca actual
    brand_data = combined_data[combined_data['SUBBRAND'] == brand]

    # Boxplot para predicciones
    boxplot = go.Box(
        x=brand_data['DATE'],
        y=brand_data['AMOUNT'],
        name='Forecast',
        marker_color='#8ecae6',
        boxmean=True,
        showlegend=True
    )

    # Graficar los valores reales como puntos
    real_values = go.Scatter(
        x=actual_spain['DATE'][actual_spain['SUBBRAND'] == brand],
        y=actual_spain['AMOUNT'][actual_spain['SUBBRAND'] == brand],
        mode='markers',
        name='Actuals',
        marker=dict(color='#457b9d', size=8),
        showlegend=True
    )

    layout = go.Layout(
        title=f'Boxplot Forecast vs Actuals for {brand}',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Sales'),
        legend=dict(title='Legend'),
        template='plotly_white'
    )

    y_max = max(actual_spain['AMOUNT'][actual_spain['SUBBRAND'] == brand].max(), brand_data['AMOUNT'].max()) * 1.1

    fig = go.Figure(data=[boxplot, real_values], layout=layout)
    fig.update_layout(yaxis_range=[0,y_max])
    fig.show()


Se observa que todas las submarcas presentan una dispersión relativamente alta en sus predicciones. Además, mientras que las predicciones para la marca de Pepsi suelen contener el valor final real, la submarca 7up muestra problemas a la hora de aportar predicciones certeras. Cabe destacar que, a pesar de que Pepsi presenta un comportamiento bueno, al tener ventas de valores bastante más altos que el resto de marcas, el rango de precisión aumenta proporcionalmente. Por lo tanto, hallar las ventas reales dentro del rango predefinido por las predicciones resulta más probable.

Una vez se ha evaluado el comportamiento global de la totalidad de las predicciones para cada mes, se procede a identificar cuáles de ellas se aproximan más al valor real.

In [29]:

for brand in combined_data['SUBBRAND'].unique():
    results = []
    # Filtrar datos para la marca actual
    actuals = actual_spain[actual_spain['SUBBRAND'] == brand]
    predictions = forecast_spain[forecast_spain['SUBBRAND'] == brand]
    for idx, row in actuals[['DATE', 'AMOUNT']].iterrows():
        real_date = row['DATE']
        real_value = row['AMOUNT']
        # Predicciones que solo se consideren aquellas hechas para real_date
        predicciones_validas = predictions[predictions['DATE'] == real_date]

        # Calcular el error absoluto entre las predicciones y el valor real
        predicciones_validas['ERROR'] = (predicciones_validas['AMOUNT'] - real_value).abs()

        # Id de la fila de la tabla de predicciones con el valor más cercano al real
        best_prediction = predicciones_validas.loc[predicciones_validas['ERROR'].idxmin()]

        # Calcular la diferencia temporal entre la predicción y el dato real
        date_diff = (real_date - best_prediction['FORECAST_DATE']).days

        results.append({
            'REAL_DATE': real_date,
            'REAL_VALUE': real_value,
            'PREDICTION': best_prediction['AMOUNT'],
            'VALUE_DIFF': best_prediction['ERROR'],
            'PREDICTION_DATE': best_prediction['FORECAST_DATE'],
            'DATE_DIFF_DAYS': date_diff
        })

    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values(by='REAL_DATE', ascending=True)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=results_df['REAL_DATE'], y=results_df['REAL_VALUE'],
                             mode='lines+markers', name='Actual Values',
                             marker=dict(color='#457b9d', size=8),
                             text=results_df['REAL_DATE'].dt.strftime('%Y-%m-%d'),  # Formato de fecha para el texto
                             hovertemplate='<b>Real Date:</b> %{text}<br><b>Real Value:</b> %{y}<extra></extra>'))

    fig.add_trace(go.Scatter(x=results_df['REAL_DATE'], y=results_df['PREDICTION'],
                             mode='lines+markers', name='Predictions',
                             marker=dict(color='#8ecae6', size=8),
                             text=results_df['PREDICTION_DATE'].dt.strftime('%Y-%m-%d'),  # Formato de fecha para el texto
                             hovertemplate='<b>Prediction Date:</b> %{text}<br><b>Value:</b> %{y}<extra></extra>'))

    # Delimitar cota superior del eje y
    y_max = max(results_df['REAL_VALUE'].max(), results_df['PREDICTION'].max()) * 1.1

    fig.update_layout(
        title=f'Forecast vs Actuals for {brand}',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Sales'),
        template='plotly_white',
        showlegend=True
    )
    fig.update_layout(yaxis_range=[0, y_max])
    fig.show()

    print(f"\nSUBMARCA - {brand}:\n", results_df.head(4))
    print('\nLas predicciones óptimas se calcularon en las siguientes fechas: ',
          [date.strftime('%Y-%m-%d') for date in results_df['PREDICTION_DATE'].unique()])
    print('\nDistancia Media: {:.2f}'.format(results_df['VALUE_DIFF'].mean()))
    print('Desviación Estándar: {:.2f}'.format(results_df['VALUE_DIFF'].std()))


SUBMARCA - Pepsi Max (L3):
     REAL_DATE     REAL_VALUE    PREDICTION    VALUE_DIFF PREDICTION_DATE  \
1  2023-01-01   83274.055497  93305.411957  10031.356460      2022-12-01   
19 2023-02-01   80772.495122  83771.577272   2999.082150      2022-12-01   
10 2023-03-01  108883.108238  98687.596793  10195.511445      2022-12-01   
0  2023-04-01   90623.557438  89660.961836    962.595602      2022-12-01   

    DATE_DIFF_DAYS  
1               31  
19              62  
10              90  
0              121  

Las predicciones óptimas se calcularon en las siguientes fechas:  ['2022-12-01']

Distancia Media: 2536.28
Desviación Estándar: 3309.98



SUBMARCA - Pepsi Regular (L3):
     REAL_DATE     REAL_VALUE     PREDICTION    VALUE_DIFF PREDICTION_DATE  \
17 2023-01-01   99292.758871  108418.891662   9126.132791      2022-12-01   
9  2023-02-01   85035.201792   95476.223334  10441.021542      2022-12-01   
4  2023-03-01  122439.036792  113240.739536   9198.297256      2022-12-01   
2  2023-04-01   99838.514061   98328.586909   1509.927152      2022-12-01   

    DATE_DIFF_DAYS  
17              31  
9               62  
4               90  
2              121  

Las predicciones óptimas se calcularon en las siguientes fechas:  ['2022-12-01']

Distancia Media: 2930.47
Desviación Estándar: 3665.87



SUBMARCA - 7up Free (L3):
     REAL_DATE    REAL_VALUE    PREDICTION   VALUE_DIFF PREDICTION_DATE  \
17 2023-01-01  54525.175586  60943.975454  6418.799868      2022-12-01   
16 2023-02-01  50344.871190  56467.241411  6122.370222      2022-12-01   
6  2023-03-01  64655.471450  69172.941141  4517.469691      2022-12-01   
11 2023-04-01  54815.016250  55546.758200   731.741950      2022-12-01   

    DATE_DIFF_DAYS  
17              31  
16              62  
6               90  
11             121  

Las predicciones óptimas se calcularon en las siguientes fechas:  ['2022-12-01']

Distancia Media: 2031.29
Desviación Estándar: 2076.47



SUBMARCA - 7up (L3):
     REAL_DATE    REAL_VALUE    PREDICTION    VALUE_DIFF PREDICTION_DATE  \
12 2023-01-01  78971.969596  52156.149274  26815.820321      2022-12-01   
18 2023-02-01  69302.973519  65063.618870   4239.354650      2022-12-01   
9  2023-03-01  94306.650948  68081.917712  26224.733236      2022-12-01   
4  2023-04-01  73141.160283  66646.508476   6494.651807      2022-12-01   

    DATE_DIFF_DAYS  
12              31  
18              62  
9               90  
4              121  

Las predicciones óptimas se calcularon en las siguientes fechas:  ['2022-12-01']

Distancia Media: 8031.48
Desviación Estándar: 9063.32



SUBMARCA - Lipton (L3):
     REAL_DATE    REAL_VALUE    PREDICTION   VALUE_DIFF PREDICTION_DATE  \
17 2023-01-01   9208.979380   7967.645287  1241.334093      2022-12-01   
14 2023-02-01  10132.837982   7424.886701  2707.951280      2022-12-01   
10 2023-03-01   9426.005681   9243.717390   182.288292      2022-12-01   
12 2023-04-01   9617.009842  11321.486196  1704.476354      2022-12-01   

    DATE_DIFF_DAYS  
17              31  
14              62  
10              90  
12             121  

Las predicciones óptimas se calcularon en las siguientes fechas:  ['2022-12-01']

Distancia Media: 1009.08
Desviación Estándar: 767.11


Con el cálculo de la distancia media entre predicciones y valores reales, se vuelve a confirmar la problemática que el modelo de predicción presenta para la submarca 7up.

De acuerdo con los resultados obtenidos, se logra encontrar un patrón bastante claro en lo que respecta a la línea temporal de las predicciones más precisas. En España, para los resultados mensuales de cada submarca, se consiguió predecir el volumen de ventas con mayor tasa de éxito en el mes de diciembre del año 2022. Así, se puede estimar que los datos recogidos hasta ese momento junto a la proyección del comportamiento comercial ofrecieron las mejores condiciones para obtener un modelo predictivo adecuado y cercano a la realidad.